# Criação de dados

A proposta deste notebook é criar um dataframe de dados mockados contendo as seguintes colunas:
- id
- data
- usuário
- empresa

In [1]:
import pandas as pd
from faker import Faker
import uuid
from datetime import datetime
from random import choice

### Imports
- **pandas**: para criar uma tabela
- **faker**: para criar os dados
- **uuid**: para chaves primárias unicas
- **datetime e random**: complementares

In [2]:
# Instanciando o Faker
fake = Faker(["pt_BR"])

In [3]:
# Quantidade de registros na tabela

lines = 500_000

In [4]:
# Criação de dados 

base_company_list = [fake.unique.company_id() for _ in range(int(1_000))] # 1k emrpesas
base_user_list = [fake.unique.email() for _ in range(int(20_000))] # 20k de usuários

# para um contexto mais lógico, onde um usuário sempre pertencer à uma mesma empresa
user_company = [(base_user, choice(base_company_list)) for base_user in base_user_list]
full_user_company = [choice(user_company) for _ in range(lines)]
 
# Criação dos dados
data = {
    "id": [uuid.uuid4() for _ in range(lines)],
    "insert_date": [fake.date_time_between(start_date=datetime(2023, 10, 1)) for _ in range(lines)],
    "username": [user[0] for user in full_user_company],
    "company": [company[1] for company in full_user_company],
}


In [5]:
# Criação do dataframe
df = pd.DataFrame(data)

In [6]:
# Apenas vendo como ficou a tabela
df.head()

,id,insert_date,username,company
0,148fc80b-b8b5-411a-8099-5c3aa94582ce,2023-10-11 15:27:55,cecilianogueira@example.com,40957263000108
1,4059d421-4042-4471-99d1-d8b4b427fffb,2023-10-25 07:43:22,caionogueira@example.net,21594708000198
2,17925564-12e1-43b3-8ace-2c26c61baa8a,2023-10-31 22:20:01,costeladiogo@example.org,31709628000194
3,47a669f9-2b0a-499f-92b0-524d01a075e9,2023-10-02 04:16:30,cavalcantimiguel@example.net,87652914000115
4,3c217c67-20be-4665-8f56-de523ebe3315,2023-10-23 01:04:00,nicoleoliveira@example.org,81350927000153


In [7]:
# Vendo os tipos das colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id           500000 non-null  object        
 1   insert_date  500000 non-null  datetime64[ns]
 2   username     500000 non-null  object        
 3   company      500000 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 15.3+ MB


In [8]:
# Gerando o arquivo SQL

# Este foi o filename que usei ao rodar no meu pc para criar o script
filename = "../docker_databases/init.sql"

# Estou deixando este apenas para que possa executar no seu computador sem comprometer o resultado
# das analises que realizei.
# filename = "../docker_databases/init_avaliacao.sql"

with open(filename, "w") as f:
    f.write("\n\n-- Criação da tabela")
    f.write("""
    CREATE TABLE t_ponto (
          id VARCHAR(255) PRIMARY KEY,
          insert_date TIMESTAMP,
          username VARCHAR(255),
          company VARCHAR(255)
    );\n
    """)
    
    f.write("\n\n-- Exportação dos dados\n\n")
    for i in df.index:
        f.write(f"INSERT INTO t_ponto (id, insert_date, username, company) VALUES ('{str(df.iloc[i, 0])}', '{df.iloc[i, 1]}', '{df.iloc[i, 2]}', '{df.iloc[i, 3]}');\n")

In [9]:
print("Fim")

Fim


## Conclusão

foi gerado um script sql para que possa ser instanciado o container com o banco ja contendo as informações. Sempre que o container for executado no composer, ele ja irá conter os dados.